In [1]:
from numpy import *
import import_ipynb
from nl_of_alarm import near,nl

importing Jupyter notebook from nl of alarm.ipynb


In [2]:
def loadDataSet():
    nlnear=[]
    for n in near:
        nnear=[]
        for x in n:
            nnear.append(nl[x])
        nlnear.append(nnear)
    return nlnear

In [3]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if [item] not in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we
                            #can use it as a key in a dict    

def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [4]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [5]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [44]:
dataSet= loadDataSet()
C1=createC1(dataSet)
D=list(map(set,dataSet))
L,suppData=apriori(dataSet,minSupport=0.02)
rules=generateRules(L, suppData, minConf=0.2)

frozenset({'元谋县022-库南-HW-MA5608T(重要)'}) --> frozenset({'4494-元谋县农村信用合作社物茂分社'}) conf: 0.20064377682403434
frozenset({'元谋县021-凉山-HW-MA5608T'}) --> frozenset({'4494-元谋县农村信用合作社物茂分社'}) conf: 0.21397379912663755
frozenset({'4468-元谋县农村信用合作社新华分社'}) --> frozenset({'4494-元谋县农村信用合作社物茂分社'}) conf: 0.21966527196652721
frozenset({'元谋县002-黄瓜园镇-HW-MA5680T(重要)'}) --> frozenset({'4494-元谋县农村信用合作社物茂分社'}) conf: 0.22601279317697226
frozenset({'元谋县001-移动公司-HW-MA5680T(重要)'}) --> frozenset({'4494-元谋县农村信用合作社物茂分社'}) conf: 0.2188841201716738
frozenset({'元谋县024-大空基站-HW-MA5608T(重要)'}) --> frozenset({'4494-元谋县农村信用合作社物茂分社'}) conf: 0.20549450549450549
frozenset({'元谋县006-老城-HW-MA5680T(重要)'}) --> frozenset({'4494-元谋县农村信用合作社物茂分社'}) conf: 0.20323741007194246


In [45]:
C1

[frozenset({'10001-元谋县局PTN7900_HX'}),
 frozenset({'10002-元谋县邮电局'}),
 frozenset({'10020-元谋县罗岔'}),
 frozenset({'10021-元谋县弯腰树'}),
 frozenset({'10026-元谋县杨柳村'}),
 frozenset({'10029-元谋县体育馆'}),
 frozenset({'10035-元谋县永武高速国兴_B'}),
 frozenset({'10038-元谋元双路4公里'}),
 frozenset({'10046-元谋县光辉_B'}),
 frozenset({'10056-元谋县水平石_B'}),
 frozenset({'10068-元谋县火车站2'}),
 frozenset({'10071-元谋县老城镇'}),
 frozenset({'10072-元谋县华丰'}),
 frozenset({'10075-元谋县物贸2_B'}),
 frozenset({'10083-元谋县新华乡政府'}),
 frozenset({'10085-元谋县平田乡政府'}),
 frozenset({'10089-元谋县芝麻箐'}),
 frozenset({'10093-元谋县公安四级网汇聚_Q'}),
 frozenset({'10098-元谋县龙山'}),
 frozenset({'10100-元谋县苴林'}),
 frozenset({'10101-元谋县挨相'}),
 frozenset({'10115-元谋县芝麻'}),
 frozenset({'10121-元谋县小乐多'}),
 frozenset({'10124-元谋县那旧'}),
 frozenset({'10125-元谋县河西'}),
 frozenset({'10126-元谋县小庄莲'}),
 frozenset({'10131-元谋县大村'}),
 frozenset({'10132-元谋县石灰村'}),
 frozenset({'10135-元谋县腊居'}),
 frozenset({'10137-元谋县那别_B'}),
 frozenset({'10139-元谋县平田培英'}),
 frozenset({'10141-元谋县大雷宰'}),
 frozenset({'1014

In [46]:
L

[[frozenset({'元谋县004-姜驿乡-HW-MA5680T(重要)'}),
  frozenset({'元谋县032-新天地-HW-MA5800-X17(重要)'}),
  frozenset({'元谋县003-物茂乡-HW-MA5680T(重要)'}),
  frozenset({'元谋县042-龙泉基站-HW-MA5800-X17(重要)'}),
  frozenset({'元谋县008-新华-HW-MA5680T(重要)'}),
  frozenset({'元谋县020-牛街基站-HW-MA5608T(重要)'}),
  frozenset({'元谋县007-平田-HW-MA5680T(重要)'}),
  frozenset({'元谋县019-五福-HW-MA5608T(重要)'}),
  frozenset({'元谋县017-红坡基站-HW-MA5608T(重要)'}),
  frozenset({'元谋县006-老城-HW-MA5680T(重要)'}),
  frozenset({'元谋县022-库南-HW-MA5608T(重要)'}),
  frozenset({'元谋县014-花同基站-HW-MA5800-X7(重要)'}),
  frozenset({'元谋县005-能禹-HW-MA5680T(重要)'}),
  frozenset({'元谋县021-凉山-HW-MA5608T'}),
  frozenset({'元谋县024-大空基站-HW-MA5608T(重要)'}),
  frozenset({'4468-元谋县农村信用合作社新华分社'}),
  frozenset({'元谋县002-黄瓜园镇-HW-MA5680T(重要)'}),
  frozenset({'元谋县036-移动楼2-HW-MA5800-X17(重要)'}),
  frozenset({'元谋县018-中兴-HW-MA5608T(重要)'}),
  frozenset({'元谋县001-移动公司-HW-MA5680T(重要)'}),
  frozenset({'4494-元谋县农村信用合作社物茂分社'})],
 [frozenset({'元谋县022-库南-HW-MA5608T(重要)', '元谋县024-大空基站-HW-MA5608T(重要)'}),
  froze

In [47]:
rules

[(frozenset({'元谋县022-库南-HW-MA5608T(重要)'}),
  frozenset({'4494-元谋县农村信用合作社物茂分社'}),
  0.20064377682403434),
 (frozenset({'元谋县021-凉山-HW-MA5608T'}),
  frozenset({'4494-元谋县农村信用合作社物茂分社'}),
  0.21397379912663755),
 (frozenset({'4468-元谋县农村信用合作社新华分社'}),
  frozenset({'4494-元谋县农村信用合作社物茂分社'}),
  0.21966527196652721),
 (frozenset({'元谋县002-黄瓜园镇-HW-MA5680T(重要)'}),
  frozenset({'4494-元谋县农村信用合作社物茂分社'}),
  0.22601279317697226),
 (frozenset({'元谋县001-移动公司-HW-MA5680T(重要)'}),
  frozenset({'4494-元谋县农村信用合作社物茂分社'}),
  0.2188841201716738),
 (frozenset({'元谋县024-大空基站-HW-MA5608T(重要)'}),
  frozenset({'4494-元谋县农村信用合作社物茂分社'}),
  0.20549450549450549),
 (frozenset({'元谋县006-老城-HW-MA5680T(重要)'}),
  frozenset({'4494-元谋县农村信用合作社物茂分社'}),
  0.20323741007194246)]

In [49]:
from py2neo import Graph,Node,Relationship

In [50]:
names=[]
for frozen in rules:
    if list(frozen[0]) not in names:
        names.append(list(frozen[0]))
    if list(frozen[1]) not in names:
        names.append(list(frozen[1]))
names

[['元谋县022-库南-HW-MA5608T(重要)'],
 ['4494-元谋县农村信用合作社物茂分社'],
 ['元谋县021-凉山-HW-MA5608T'],
 ['4468-元谋县农村信用合作社新华分社'],
 ['元谋县002-黄瓜园镇-HW-MA5680T(重要)'],
 ['元谋县001-移动公司-HW-MA5680T(重要)'],
 ['元谋县024-大空基站-HW-MA5608T(重要)'],
 ['元谋县006-老城-HW-MA5680T(重要)']]

In [66]:
relation=[]
for frozen in rules:
    relation.append([list(frozen[0]),frozen[2],list(frozen[1])])
relation

[[['元谋县022-库南-HW-MA5608T(重要)'], 0.20064377682403434, ['4494-元谋县农村信用合作社物茂分社']],
 [['元谋县021-凉山-HW-MA5608T'], 0.21397379912663755, ['4494-元谋县农村信用合作社物茂分社']],
 [['4468-元谋县农村信用合作社新华分社'], 0.21966527196652721, ['4494-元谋县农村信用合作社物茂分社']],
 [['元谋县002-黄瓜园镇-HW-MA5680T(重要)'],
  0.22601279317697226,
  ['4494-元谋县农村信用合作社物茂分社']],
 [['元谋县001-移动公司-HW-MA5680T(重要)'], 0.2188841201716738, ['4494-元谋县农村信用合作社物茂分社']],
 [['元谋县024-大空基站-HW-MA5608T(重要)'],
  0.20549450549450549,
  ['4494-元谋县农村信用合作社物茂分社']],
 [['元谋县006-老城-HW-MA5680T(重要)'], 0.20323741007194246, ['4494-元谋县农村信用合作社物茂分社']]]

In [67]:
graph = Graph('http://localhost:7474',username='neo4j',password='010419')

In [84]:
nam0=Node('WY',name='KN')
graph.create(nam0)
nam1=Node('WY',name='WMFS')
graph.create(nam1)
nam2=Node('WY',name='NS')
graph.create(nam2)
nam3=Node('WY',name='XHFS')
graph.create(nam3)
nam4=Node('WY',name='HGYZ')
graph.create(nam4)
nam5=Node('WY',name='YDGS')
graph.create(nam5)
nam6=Node('WY',name='DKJZ')
graph.create(nam6)
nam7=Node('WY',name='LC')
graph.create(nam7)

In [85]:
relat0=Relationship(nam0,'0.20064377682403434',nam1)
graph.create(relat0)
relat1=Relationship(nam2,'0.21397379912663755',nam1)
graph.create(relat1)
relat2=Relationship(nam3,'0.21966527196652721',nam1)
graph.create(relat2)
relat3=Relationship(nam4,'0.22601279317697226',nam1)
graph.create(relat3)
relat4=Relationship(nam5,'0.2188841201716738',nam1)
graph.create(relat4)
relat5=Relationship(nam6,'0.20549450549450549',nam1)
graph.create(relat5)
relat6=Relationship(nam7,'0.20323741007194246',nam1)
graph.create(relat6)

In [86]:
print(graph)

<Graph database=<Database uri='http://localhost:7474' secure=False user_agent='py2neo/4.3.0 urllib3/1.24.3 Python/3.8.2-final-0 (win32)'> name='data'>
